In [2]:
import os
import pydicom
import pandas as pd
from tqdm import tqdm

In [64]:
root_input = r'F:\huashan\filter_data\qingdao'
series_df = pd.DataFrame()

for case in tqdm(os.listdir(root_input)):
    cor_fs, sag_fs, ax_fs, cor_t1, cor_t2, others = [[],[],[],[],[],[]]
    total = 0
    series_folders = os.listdir(os.path.join(root_input, case))
    for series_folder in series_folders:
        series_dicom = os.path.join(root_input, case, series_folder, os.listdir(os.path.join(root_input, case, series_folder))[0])
        try:
            ds = pydicom.dcmread(series_dicom)
            series_description = ds.SeriesDescription
        except Exception as e:
            continue
        total += 1
        series_desc = series_description.lower()
        series_desc = series_desc.replace(' ','_')
        series_desc = series_desc.replace('.','_')
        series_desc = series_desc.replace(',','_')
        series_desc = series_desc.replace('-','_')
        series_words = series_desc.split('_')

        if 'fs' in series_words or 'stir' in series_words or 'spair' in series_words or 'fat' in series_words:
            if 'cor' in series_words or 'ocor' in series_words or 'co' in series_words:
                cor_fs.append(series_description)
            elif 'sag' in series_words or 'osag' in series_words or 'sg' in series_words:
                sag_fs.append(series_description)
                
            elif 'ax' in series_words or 'oax' in series_words or'tra' in series_words or 'axi' in series_words:
                ax_fs.append(series_description)
            else:
                others.append(series_description)
        elif 'cor' in series_words or 'ocor' in series_words or 'co' in series_words:
            if 't1' in series_words or 't1wi' in series_words:
                cor_t1.append(series_description)
            elif 't2' in series_words or 't2wi' in series_words:
                cor_t2.append(series_description)
            else:
                others.append(series_description)
        else:
            others.append(series_description)
        
    row_info = {'PID': case, 'cor_fs': cor_fs, 'sag_fs':sag_fs, 'ax_fs':ax_fs, 'cor_t1':cor_t1, 'cor_t2':cor_t2, 'others': others, 'total':total}
    series_df = pd.concat([series_df, pd.DataFrame([row_info])], ignore_index=True)       


100%|████████████████████████████████████████████████████████████████████████████████| 558/558 [00:37<00:00, 14.75it/s]


In [65]:
series_df.to_excel('qingdao_series.xlsx', index=False)

In [78]:
fs_cor, fs_sag, fs_ax, t1_cor = [],[],[],[]


for index, row in series_df.iterrows():
    try:
        if row.cor_fs[0] not in fs_cor:
            fs_cor.append(row.cor_fs[0])
    except Exception as e:
        print(index)
    if row.sag_fs[0] not in fs_sag:
        fs_sag.append(row.sag_fs[0])
    if row.ax_fs[0] not in fs_ax:
        fs_ax.append(row.ax_fs[0])
    try:
        if row.cor_t1[0] not in t1_cor:
            t1_cor.append(row.cor_t1[0])
    except Exception as e:
        print(index, row.PID)
print(fs_cor, fs_sag, fs_ax, t1_cor)

2 1327529
52 2630675
57 2657856
63 2767542
71 2834958
109 3159723
200 4162189
202 4214693
224 4457035
274 6152314
278 6167009
334
334 6444082
398 6556694
543 7046794
552 7096484
553 7097067
557 7105460
['COR Fat-Sat-FSE-T2WI', 'COR_SPAIR_FSE_T2WI', 'COR Fat-Sat-FSE T2WI', 'OCor fs T2', 'OCor fs T2 R', 'COR FS FSE T2WI', 'COR_FSE_FS_T2WI', 'COR_FSE_FS_T2WI--L', 'COR Fat-Sat-TSE T2WI', 'COR_FSE_FS_T2WI_R', 'COR_FSE_FS_T2WI_', 'L-COR Fat-Sat-FSE T2WI-LEFT', 'L-COR Fat-Sat-FSE T2WI', 'R-COR Fat-Sat-FSE T2WI-RIGHT', 'COR FS FSE T2WI-L', 'COR Fat-Sat-FSE T2WI-LEFT', 'COR_SPAIR_FSE_T2WI-R', 'OCor fs T2 L', 'OCor fs T2RR', 'L COR Fat-Sat-TSE T2WI', 'R-COR Fat-Sat-FSE T2WI', 'L COR_SPAIR_FSE_T2WI', 'COR Fat-Sat-FSE T2WI-RIGHT', 'R COR FS-FSE T2WI', 'OCor fs T2  L', 'COR_FSE_FS_T2WI R'] ['SAG Fat-Sat-FSE-T2WI', 'SAG_Fat-Sat_FSE_PDWI', 'SAG Fat-Sat-TSE PDWI', 'SAG Fat-Sat-FSE PDWI', 'OSag PD fs FSE', 'OSag fs T2 R', 'OSag fs T2', 'SAG FS FSE PDWI', 'SAG  pd_tse_fs', 'SAG_FSE_FS_T2WI--L', 'SAG_FSE

In [3]:
root_input = r'F:\huashan\filter_data\anning'
output_root = r'F:\huashan\preprocessed_data'
name_set = set()

for case in tqdm(os.listdir(root_input)):
    flag = False
    for series_folder in os.listdir(os.path.join(root_input, case)):
        output_folder = os.path.join(output_root, case, series_folder)
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        for series_file in os.listdir(os.path.join(root_input, case, series_folder)):
            try:
                ds = pydicom.dcmread(os.path.join(root_input,case, series_folder, series_file))
                ds.SpecificCharacterSet = 'GB18030'
                p_name = str(ds.PatientName)
                if p_name not in name_set:
                    name_set.add(p_name)
                    flag = True
                if p_name.startswith('NO'):
                    p_name = 'NO NAME'
                ds.PatientName = p_name.replace('_',' ').split('^')[0].split('=')[0]
                if flag:
                    #print(f'new_name {ds.PatientName}, pid {case}')
                    flag = False
                
                ds.save_as(os.path.join(output_folder, series_file))
            except Exception as e:
                print(e,case)
                continue

            

  0%|                                                                                          | 0/472 [00:00<?, ?it/s]C:\Users\z004b0je\AppData\Local\miniconda3\envs\work\lib\site-packages\pydicom\charset.py:714: UserWarning: Value 'GB18030' cannot be used as code extension, ignoring it
  py_encodings = _handle_illegal_standalone_encodings(
C:\Users\z004b0je\AppData\Local\miniconda3\envs\work\lib\site-packages\pydicom\charset.py:486: UserWarning: Found unknown escape sequence in encoded string value - using encoding latin_1
  warnings.warn(msg + f" - using encoding {encodings[0]}")
C:\Users\z004b0je\AppData\Local\miniconda3\envs\work\lib\site-packages\pydicom\valuerep.py:443: UserWarning: Invalid value for VR UI: '1.2.840.113704.7.32.05.2.40.38441.202308041849008980624891.0.0.0'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
  1%|▊                                                       

In [57]:
wrong_ds = ds
wrong_ds2 = pydicom.dcmread(r'F:\huashan\filter_data\anning\1002088\pd_tse_fs_cor_fil_1.2.840.113704.7.32.0.2.40.38441.2023080418520086017925211.0.0.0\I1000000')
correct_ds = pydicom.dcmread(r'F:\huashan\filter_data\anning\1673059\pd_tse_fs_cor_fil_1.2.840.113704.7.32.0.2.40.38441.2023111520140821987345919.0.0.0\I1000000')
correct_ds2 = pydicom.dcmread(r'F:\huashan\filter_data\anning\1026784\pd_tse_fs_cor_fil_1.2.840.113704.7.32.0.2.40.38441.2023072109550970007329700.0.0.0\I1000000')

In [60]:
wrong_value,correct_value = [],[]

for tag in wrong_ds:
    wrong_value.append(tag)
for tag in correct_ds:
    correct_value.append(tag)

ds_keys = ds.keys()
for ds_key in ds_keys:
    try:
        if not wrong_ds[ds_key] == correct_ds2[ds_key]:
            print(ds_key, wrong_ds[ds_key], correct_ds2[ds_key])
    
    except Exception as e:
        continue
    

(0008, 0005) (0008, 0005) Specific Character Set              CS: ['ISO_IR 100', 'GB18030'] (0008, 0005) Specific Character Set              CS: 'GB18030'
(0008, 0012) (0008, 0012) Instance Creation Date              DA: '20231205' (0008, 0012) Instance Creation Date              DA: '20231203'
(0008, 0013) (0008, 0013) Instance Creation Time              TM: '163238' (0008, 0013) Instance Creation Time              TM: '034317'
(0008, 0018) (0008, 0018) SOP Instance UID                    UI: 1.2.840.113704.7.1.0.11222110613544215.1701765145.13763 (0008, 0018) SOP Instance UID                    UI: 1.2.840.113704.7.1.0.12113120170238128.1701546159.21394
(0008, 0020) (0008, 0020) Study Date                          DA: '20231203' (0008, 0020) Study Date                          DA: '20230721'
(0008, 0021) (0008, 0021) Series Date                         DA: '20231203' (0008, 0021) Series Date                         DA: '20230721'
(0008, 0022) (0008, 0022) Acquisition Date            

In [45]:
ds.keys()

dict_keys([(0008, 0005), (0008, 0008), (0008, 0012), (0008, 0013), (0008, 0016), (0008, 0018), (0008, 0020), (0008, 0021), (0008, 0022), (0008, 0023), (0008, 0030), (0008, 0031), (0008, 0032), (0008, 0033), (0008, 0050), (0008, 0060), (0008, 0061), (0008, 0070), (0008, 0080), (0008, 0081), (0008, 0090), (0008, 1010), (0008, 1030), (0008, 1032), (0008, 103e), (0008, 1050), (0008, 1090), (0008, 1140), (0008, 2112), (0010, 0010), (0010, 0020), (0010, 0021), (0010, 0030), (0010, 0040), (0010, 1010), (0010, 1020), (0010, 1030), (0018, 0020), (0018, 0021), (0018, 0022), (0018, 0023), (0018, 0024), (0018, 0025), (0018, 0050), (0018, 0080), (0018, 0081), (0018, 0083), (0018, 0084), (0018, 0085), (0018, 0086), (0018, 0087), (0018, 0088), (0018, 0089), (0018, 0091), (0018, 0093), (0018, 0094), (0018, 0095), (0018, 1000), (0018, 1020), (0018, 1030), (0018, 1074), (0018, 1251), (0018, 1310), (0018, 1312), (0018, 1314), (0018, 1315), (0018, 1316), (0018, 1318), (0018, 5100), (0019, 0010), (0020, 00

In [56]:
set(ds.keys()) - set(correct_ds2.keys())

set()